# Final Report Without Augmentation

In [7]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw



# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils

from samples.idvscript import main
from samples import ivddataset

# Path to trained weights file
COCO_WEIGHTS_PATH = "mask_rcnn_coco.h5"

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

# Experiment Configuration

In [8]:
class IDVConfig(Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "idv"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 2

    
    # Number of classes (including background)
    NUM_CLASSES = 1 + 3  # Background + balloon

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 50
    
    #RPN_ANCHOR_SCALES = (4,8,32, 64, 128)
    
    # Loss weights for more precise optimization.
    # Can be used for R-CNN training setup.
    '''LOSS_WEIGHTS = {
        "rpn_class_loss": 1.,
        "rpn_bbox_loss": 1.,
        "mrcnn_class_loss": 1.,
        "mrcnn_bbox_loss": 1.,
        "mrcnn_mask_loss": 1.
    }'''

    # Skip detections with < 60% confidence
    DETECTION_MIN_CONFIDENCE = 0.6

config = IDVConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.6
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                16
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

# Instantiate Model

In [9]:
model = modellib.MaskRCNN(mode="training", config=config, model_dir=DEFAULT_LOGS_DIR)

# Load pretrained weights
model_path = COCO_WEIGHTS_PATH
model.load_weights(model_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
          "mrcnn_bbox", "mrcnn_mask"])

# Load Training and Validation Dataset

In [10]:
# Load training dataset
dataset_train = ivddataset.IDVDataset()
dataset_train.load_idv(dataset_dir=os.path.join("train"))
dataset_train.prepare()

# Validation dataset
dataset_val = ivddataset.IDVDataset()
dataset_val.load_idv(dataset_dir=os.path.join("val"))
dataset_val.prepare()

# Train

In [11]:
# *** This training schedule is an example. Update to your needs ***

# Training - Stage 1
print("Training network heads")
model.train(dataset_train,dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=30,
            layers='heads')

# Training - Stage 2
# Finetune layers from ResNet stage 4 and up
print("Fine tune Resnet stage 4 and up")
model.train(dataset_train,dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=40,
            layers='4+')

# Training - Stage 3
# Fine tune all layers
print("Fine tune all layers")
model.train(dataset_train,dataset_val,
            learning_rate=config.LEARNING_RATE / 10,
            epochs=50,
            layers='all')

Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: /volumes/notebooks/Final IDV/logs/idv20180622T1351/mask_rcnn_idv_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (Tim

/opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:98: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/lib/python3.6/site-packages/keras/engine/training.py:2087: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/30
49/50 [============================>.] - ETA: 2s - loss: 1.7194 - rpn_class_loss: 0.0062 - rpn_bbox_loss: 0.2549 - mrcnn_class_loss: 0.2280 - mrcnn_bbox_loss: 0.6686 - mrcnn_mask_loss: 0.5617

/opt/conda/lib/python3.6/site-packages/keras/engine/training.py:2330: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


50/50 [==============================] - 207s 4s/step - loss: 1.7004 - rpn_class_loss: 0.0061 - rpn_bbox_loss: 0.2501 - mrcnn_class_loss: 0.2241 - mrcnn_bbox_loss: 0.6668 - mrcnn_mask_loss: 0.5533 - val_loss: 1.1845 - val_rpn_class_loss: 0.0107 - val_rpn_bbox_loss: 0.2225 - val_mrcnn_class_loss: 0.1104 - val_mrcnn_bbox_loss: 0.4833 - val_mrcnn_mask_loss: 0.3575
Epoch 2/30
50/50 [==============================] - 186s 4s/step - loss: 0.9371 - rpn_class_loss: 0.0070 - rpn_bbox_loss: 0.1639 - mrcnn_class_loss: 0.0773 - mrcnn_bbox_loss: 0.4125 - mrcnn_mask_loss: 0.2763 - val_loss: 0.9816 - val_rpn_class_loss: 0.0092 - val_rpn_bbox_loss: 0.2478 - val_mrcnn_class_loss: 0.0716 - val_mrcnn_bbox_loss: 0.3677 - val_mrcnn_mask_loss: 0.2853
Epoch 3/30
50/50 [==============================] - 185s 4s/step - loss: 0.7784 - rpn_class_loss: 0.0053 - rpn_bbox_loss: 0.1571 - mrcnn_class_loss: 0.0531 - mrcnn_bbox_loss: 0.3009 - mrcnn_mask_loss: 0.2621 - val_loss: 0.9104 - val_rpn_class_loss: 0.0080 - val

Epoch 23/30
50/50 [==============================] - 179s 4s/step - loss: 0.2693 - rpn_class_loss: 0.0039 - rpn_bbox_loss: 0.0316 - mrcnn_class_loss: 0.0321 - mrcnn_bbox_loss: 0.0653 - mrcnn_mask_loss: 0.1364 - val_loss: 0.7496 - val_rpn_class_loss: 0.0043 - val_rpn_bbox_loss: 0.1844 - val_mrcnn_class_loss: 0.0627 - val_mrcnn_bbox_loss: 0.2016 - val_mrcnn_mask_loss: 0.2966
Epoch 24/30
50/50 [==============================] - 179s 4s/step - loss: 0.2870 - rpn_class_loss: 0.0023 - rpn_bbox_loss: 0.0384 - mrcnn_class_loss: 0.0283 - mrcnn_bbox_loss: 0.0704 - mrcnn_mask_loss: 0.1476 - val_loss: 0.7875 - val_rpn_class_loss: 0.0044 - val_rpn_bbox_loss: 0.2269 - val_mrcnn_class_loss: 0.0517 - val_mrcnn_bbox_loss: 0.1929 - val_mrcnn_mask_loss: 0.3115
Epoch 25/30
50/50 [==============================] - 179s 4s/step - loss: 0.2730 - rpn_class_loss: 0.0026 - rpn_bbox_loss: 0.0347 - mrcnn_class_loss: 0.0297 - mrcnn_bbox_loss: 0.0642 - mrcnn_mask_loss: 0.1418 - val_loss: 0.7367 - val_rpn_class_loss

Epoch 31/40
50/50 [==============================] - 250s 5s/step - loss: 0.2805 - rpn_class_loss: 0.0027 - rpn_bbox_loss: 0.0552 - mrcnn_class_loss: 0.0371 - mrcnn_bbox_loss: 0.0720 - mrcnn_mask_loss: 0.1136 - val_loss: 0.7781 - val_rpn_class_loss: 0.0022 - val_rpn_bbox_loss: 0.1451 - val_mrcnn_class_loss: 0.0949 - val_mrcnn_bbox_loss: 0.2178 - val_mrcnn_mask_loss: 0.3180
Epoch 32/40
50/50 [==============================] - 212s 4s/step - loss: 0.3318 - rpn_class_loss: 0.0028 - rpn_bbox_loss: 0.0844 - mrcnn_class_loss: 0.0368 - mrcnn_bbox_loss: 0.0805 - mrcnn_mask_loss: 0.1272 - val_loss: 0.7035 - val_rpn_class_loss: 0.0027 - val_rpn_bbox_loss: 0.1518 - val_mrcnn_class_loss: 0.0627 - val_mrcnn_bbox_loss: 0.1729 - val_mrcnn_mask_loss: 0.3135
Epoch 33/40
50/50 [==============================] - 213s 4s/step - loss: 0.3043 - rpn_class_loss: 0.0023 - rpn_bbox_loss: 0.0635 - mrcnn_class_loss: 0.0327 - mrcnn_bbox_loss: 0.0695 - mrcnn_mask_loss: 0.1363 - val_loss: 0.7211 - val_rpn_class_loss

Epoch 41/50
50/50 [==============================] - 276s 6s/step - loss: 0.1681 - rpn_class_loss: 0.0019 - rpn_bbox_loss: 0.0239 - mrcnn_class_loss: 0.0157 - mrcnn_bbox_loss: 0.0317 - mrcnn_mask_loss: 0.0948 - val_loss: 0.6019 - val_rpn_class_loss: 0.0027 - val_rpn_bbox_loss: 0.1417 - val_mrcnn_class_loss: 0.0385 - val_mrcnn_bbox_loss: 0.1475 - val_mrcnn_mask_loss: 0.2715
Epoch 42/50
50/50 [==============================] - 236s 5s/step - loss: 0.1354 - rpn_class_loss: 9.0348e-04 - rpn_bbox_loss: 0.0143 - mrcnn_class_loss: 0.0108 - mrcnn_bbox_loss: 0.0235 - mrcnn_mask_loss: 0.0858 - val_loss: 0.6294 - val_rpn_class_loss: 0.0026 - val_rpn_bbox_loss: 0.1524 - val_mrcnn_class_loss: 0.0435 - val_mrcnn_bbox_loss: 0.1458 - val_mrcnn_mask_loss: 0.2852
Epoch 43/50
50/50 [==============================] - 235s 5s/step - loss: 0.1310 - rpn_class_loss: 0.0011 - rpn_bbox_loss: 0.0157 - mrcnn_class_loss: 0.0128 - mrcnn_bbox_loss: 0.0223 - mrcnn_mask_loss: 0.0791 - val_loss: 0.6268 - val_rpn_class_